In [26]:
import numpy as np
c = np.array([2, -3, 4])
A = np.array([[1, 1, 1], [2, 1, 0], [0, 3, 1]])
b = np.array([3, 5, 4])

In [39]:
tol = 1e-7

def tableau_phase1(T,n,m,basis):
    while True:
        j = np.argmin(T[-1,:-1])
        if T[-1,j] >= -tol:
            # optimal solution found
            x = np.zeros(n)
            # TODO
            # x[basis < n] = T[basis < n, -1]
            return T,basis, x
        # step 2: select leaving variable
        # T[:-1,-1]/T[:-1,j]
        
        min_index = -1
        min_ratio = 1e9
        for it in range(m):
            if T[it,j] > 0:
                if T[it,-1]/T[it,j] < min_ratio:
                    min_index = it
                    min_ratio = T[it,-1]/T[it,j] 
        
        # i = np.argmin(T[:-1,-1]/T[:-1,j]) #Doubt about correctness of this
        
        i = min_index
        
        if min_index == -1:
            # unbounded solution
            return None, None,None
        
        
        # step 3: perform pivot
        basis[i] = j
        
        T[i,:] /= T[i,j]
        for k in range(m+1):
            if k != i:
                T[k,:] -= T[k,j] * T[i,:]


def tableau_phase2(T,n,m,basis):
    while True:
        j = np.argmin(T[-1,:-1])
        if T[-1,j] >= -tol:
            # optimal solution found
            x = np.zeros(n)
            # TODO
            # x[basis < n] = T[basis < n, -1]
            return T,basis, x
        # step 2: select leaving variable
        # T[:-1,-1]/T[:-1,j]
        
        min_index = -1
        min_ratio = 1e9
        for it in range(m):
            if T[it,j] > 0:
                if T[it,-1]/T[it,j] < min_ratio:
                    min_index = it
                    min_ratio = T[it,-1]/T[it,j] 
        
        # i = np.argmin(T[:-1,-1]/T[:-1,j]) #Doubt about correctness of this
        
        i = min_index
        
        if min_index == -1:
            # unbounded solution
            return None, None,None
        
        
        # step 3: perform pivot
        basis[i] = j
        
        T[i,:] /= T[i,j]
        for k in range(m+1):
            if k != i:
                T[k,:] -= T[k,j] * T[i,:]


In [42]:
m, n_old = A.shape
n = n_old + m
# Converting to standard form Ax = b
A = np.hstack((A, np.eye(A.shape[0])))
c = np.hstack((c, np.zeros(A.shape[0])))
# Minimising c
c = -c

# making b positive
for i in range(m):
    if(b[i] < 0):
        b[i] = -b[i]
        A[i,:] = -A[i,:]

# Phase1
# Ax + Iy = b
c_phase1 = np.concatenate((np.zeros(n), np.ones(m)))
A_phase1 = np.hstack((A, np.eye(m)))
basis_indices = np.arange(n, n+m)

# Constructing initial tableau
T = np.zeros((m+1, n+m+1))
# T[1:, 1:] = A
# T[1:,0] = b
# T[0, 1:] = c_phase1 - np.ones(m).T @ A
# T[0,0] = -np.ones(m) @ b
T[:-1, :-1] = A_phase1
T[0:-1:,-1] = b
T[-1, :-1] = c_phase1 - np.ones(m).T @ A_phase1
T[-1,-1] = -np.ones(m) @ b

# Solving the initial tableau
T, basis_indices, x = tableau_phase1(T, n, m, basis_indices)

if(abs(T[-1,-1]) > tol):
    
    print("F Cost not 0 ")

T = np.hstack((T[:,:n],T[:,n+m:]))
c_b = c[basis_indices]
T[-1, :-1] = c - c_b @ T[:-1, :-1]
T[-1,-1] = -c_b @ T[0:-1:,-1]


T, basis_indices, x = tableau_phase2(T, n, m, basis_indices)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 4 but corresponding boolean dimension is 3

In [30]:
T

array([[  1.,   1.,   1.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,
          0.,   3.],
       [  2.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,
          0.,   5.],
       [  0.,   3.,   1.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          1.,   4.],
       [ -3.,  -5.,  -2.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1.,   0.,   0.,
          0., -12.]])

In [31]:
b

array([3, 5, 4])

In [7]:
A = np.ones((10,3))

In [18]:
b @ A

array([10., 10., 10.])

In [19]:
b@b

10.0

In [23]:
a = np.array([1,2,3])

In [24]:
c = np.array([3,4,0])

In [25]:
a/c

/tmp/ipykernel_750889/749814572.py:1: RuntimeWarning: divide by zero encountered in true_divide
  a/c


array([0.33333333, 0.5       ,        inf])

In [33]:
basis = np.array([3,2,0])
a = np.array([7,1,2,3,4,5,6,7,8])
a[basis]

array([3, 2, 7])

In [37]:
a < 3

array([False,  True,  True, False, False, False, False, False, False])

In [38]:
a[a<3]

array([1, 2])